# VC - Entrega Práctica 3
---
 - Luis Martín Pérez
- Andrea Santana López

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import csv

from scipy.stats import cosine

### TAREA 1 - Detector de dinero en monedas

In [6]:
import cv2
umbral = 200
img = cv2.imread('CoinsImages/Monedas.jpg')
img_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th,img_th = cv2.threshold(img_gris,umbral,255,cv2.THRESH_BINARY_INV)

click_flag = False
gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#Suaviza imagen (elimina altas frecuencias)
pimg = cv2.medianBlur(gris, 7)

circ = cv2.HoughCircles(
        pimg,  # imagen
        cv2.HOUGH_GRADIENT,  # tipo de detección
        1,
        100,  # distancia mínima entre círculos
        param1=100, # valor del gradiente
        param2=50, # umbral acumulador
        minRadius=50,  # radio mínimo
        maxRadius=150,  # radio máximo
    )

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = 2

money_amount = 0.0
started_count = False
reference_coin_value = 1.0

def identify_coin_value_from_reference_coin(selected_coin_radius, current_coin_radius):
    global reference_coin_value
    if selected_coin_radius == 0:
        return 0.
    size_ratio = current_coin_radius * 2 / float(selected_coin_radius*2)
    scale = float(selected_coin_radius) / min(img.shape[1], img.shape[0])
    if 0.98 <= current_coin_radius / selected_coin_radius <= 1.05:
        return 1.0
    if size_ratio <= 0.66 + scale * current_coin_radius / current_coin_radius:
        return 0.01
    elif size_ratio <= 0.701 + scale * current_coin_radius / current_coin_radius:
        return 0.02
    elif size_ratio <= 0.74 + scale * current_coin_radius / current_coin_radius:
        return 0.1
    elif size_ratio <= 0.8 + scale * current_coin_radius / current_coin_radius:
        return 0.05
    elif size_ratio <= 0.8501 + scale * current_coin_radius / current_coin_radius:
        return 0.2
    elif size_ratio <= 0.95 + scale * current_coin_radius / current_coin_radius:
        return 0.5
    elif size_ratio <= 1.3 + scale * current_coin_radius / current_coin_radius:
        return 2.0
    return 0.

calculated = False
def calculate_coin_amounts(coin_circles, selected_coin_radius):
    global money_amount, calculated
    calculated = True
    for circle_x, circle_y, circle_radius in coin_circles:
        cv2.circle(img, (int(circle_x), int(circle_y)), int(circle_radius*0.3), (0, 30, 240), -1)
        cv2.circle(img, (int(circle_x), int(circle_y)), int(circle_radius*0.3 + 1), (0, 0, 0), 3)
        current_coin_value = identify_coin_value_from_reference_coin(selected_coin_radius, circle_radius)
        money_amount += current_coin_value
        cv2.putText(img, f'Coin: {current_coin_value:.2f}$', (int(circle_x - circle_radius*1.1) , int(circle_y + circle_radius*1.2)), font,
                   fontScale, (255, 0, 0), thickness, cv2.LINE_AA)
    cv2.putText(img, f"Total money amount is: {money_amount:.2f}$", (50, 50), font, fontScale, (255, 0, 0), thickness, cv2.LINE_AA)
    print(f"Total money amount is: {money_amount:.2f}$")


selected_coin_radius = 0
def start_count_on_mouse_click(event, x, y, flags, param):
    global circ, started_count, selected_coin_radius, calculated
    if event == cv2.EVENT_LBUTTONDOWN and not started_count:
        started_count, selected_coin_radius = select_reference_coin(circ[0], x, y, param)
    if started_count and not calculated:
        calculate_coin_amounts(circ[0], selected_coin_radius)


def is_in_bounds(circle_x, circle_y, click_x, click_y, radius):
    return (click_x - circle_x)**2 + (click_y - circle_y)**2 <= radius**2


def select_reference_coin(coin_circles, click_x, click_y, img):
    for circle_x, circle_y, circle_radius in coin_circles:
        if is_in_bounds(circle_x, circle_y, click_x, click_y, circle_radius):
            cv2.putText(img, 'You clicked this', (click_x, click_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            return True, circle_radius
    return False, 0


window_name = "Monedas"
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, start_count_on_mouse_click, img)

while True:
    cv2.imshow(window_name, img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

Total money amount is: 5.18$


# Detector Microplasticos

In [ ]:
from sklearn.naive_bayes import GaussianNB

#Carga imagen directamente en grises
imgorig = cv2.imread('./MicroplasticImages/MPs_test.jpg', cv2.IMREAD_GRAYSCALE)

img = cv2.GaussianBlur(imgorig,(5,5),0)

#Umbralizados
ret,imth1 = cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
thotsu,imth2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
print('Umbral escogido ', thotsu)
imth3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
imth4 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original', 'Fijo','Otsu th='+str(int(thotsu)),
            'Adaptivo promedio', 'Adaptivo Gaussiano']
images = [img, imth1, imth2, 255 - imth3, 255 - imth4]

for i in range(5):
    plt.subplot(2,5,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i], fontsize=7)
    plt.xticks([]),plt.yticks([])

    #Obtiene únicamente los contornos externos
    if i>0:
        res,imth = cv2.threshold(images[i],120,255,cv2.THRESH_BINARY)
        contornos, hierarchy= cv2.findContours(imth,
        cv2.RETR_EXTERNAL ,
        cv2.CHAIN_APPROX_SIMPLE)
        img_cont = np.zeros(img.shape)
        cv2.drawContours(img_cont, contornos, -1, (255,255,255), -1)
        plt.subplot(2,5,i+6),plt.imshow(img_cont,'gray')
        plt.xticks([]),plt.yticks([])
plt.show()


#Segunda parte: Guardar contornos en CSV
# Imagen y anotaciones
imagen = "./MicroplasticImages/MPs_test.jpg"          # Imagen original
csv_file = "./MicroplasticCSV/MPs_test_bbs.csv"  # CSV con coordenadas y tipología

# Colores de cada clase
colores = {
    "FRA": (0, 0, 255),   # Rojo
    "PEL": (0, 255, 0),   # Verde
    "TAR": (255, 0, 0)    # Azul
}

# Imagen
img = cv2.imread(imagen)

# Cara csv y dibujar rectángulos
with open(csv_file, newline="") as file:
    reader = csv.DictReader(file)
    for row in reader:
        etiqueta = row["label"]
        x_min, y_min, x_max, y_max = map(int, [row["x_min"], row["y_min"], row["x_max"], row["y_max"]])

        # Color según etiqueta
        color = colores.get(etiqueta, (0, 0, 0))  # negro por defecto si no encuentra

        # Dibujar rectángulo
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)

        # Etiqueta
        cv2.putText(img, etiqueta, (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Visualiza resultado
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Anotaciones")
plt.axis('off')
plt.show()

#Cargamos tres subimágenes, por simplicidad, de cada uno de los tres tipos considerados (el alquitrán no es microplástico)
imgF = cv2.imread('./MicroplasticImages/FRA.png')
imgP = cv2.imread('./MicroplasticImages/PEL.png')
imgT = cv2.imread('./MicroplasticImages/TAR.png')

#Mostramos
plt.subplot(131)
plt.axis("off")
plt.imshow(imgF)
plt.title('Fragmentos')
plt.subplot(132)
plt.axis("off")
plt.imshow(imgP)
plt.title('Pellet')
plt.subplot(133)
plt.axis("off")
plt.imshow(imgT)
plt.title('Alquitrán')


import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


# Numero de muestras
n = 100
nclases = 4

# A falta de clasificador y conjunto de datos, creamos anotaciones y predicciones de forma aleatoria
# Vector aleatorio con etiquetas anotadas
y = [random.randint(0, nclases - 1) for _ in range(n)]
print('Anotaciones ' , y)

# Vector aleatorio con etiquetas predichas por un supuesto clasificador
y_pred = [random.randint(0, nclases - 1) for _ in range(n)]
print('Predicciones ' , y_pred)

print('¿Cómo de bien encajan anotación y predicción?')

#Cálculo de métricas
accuracy = accuracy_score(y, y_pred)
#Para más de una clase se define la forma de promediar
precision = precision_score(y, y_pred,average='weighted')
recall = recall_score(y, y_pred,average='weighted')
f1score = f1_score(y, y_pred,average='weighted')

print(f"Accuracy (TP/(n))= {accuracy}")
print(f"Precision (TP/(TP+FP)) = {precision}")
print(f"Recall (TP/(TP+FN)) = {recall}")
print(f"F1 Score (2*(precision*recall)/(precision+recall)) = {f1score}")


conf_matrix = confusion_matrix(y, y_pred)
plt.figure(figsize=(8,8))
sns.set(font_scale = 1.75)#tamaños tipografía
sns.set(font_scale = 3.0)

ax = sns.heatmap(
        conf_matrix, # confusion matrix 2D array
        annot=True, # Muestra números en las celdas
        fmt='d', # valores enteros
        cbar=False, # sin barra de colores
        cmap='flag', # mapa de colores
        #vmax=175 # contraste de color
    )

#Etiquetas matriz de confusión
label_font = {'size':'25'}
ax.set_xlabel("Predicha", labelpad=-0.75, fontdict=label_font)
ax.set_ylabel("Real/Anotado", labelpad=20, fontdict=label_font)

#https://www.kaggle.com/code/prashant111/naive-bayes-classifier-in-python#12.-Model-training-
#Faltan cosas
#MI parte
"""
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Número de etiquetas mal clasificadas: %d" % (y_test != y_pred).sum())
print("Precisión del clasificador: %.2f" % accuracy_score(y_test, y_pred))
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))
y_test.value_counts()
cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'],
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

#Aqui falta caracteristicas geometricas y de color

"""